In [1]:
import os

os.chdir("../")

%pwd

'c:\\Users\\P52s\\Documents\\Python Project\\ElasticNet_Model'

In [2]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [3]:
from WineX.constants import *
from WineX.utils.common import read_yaml, create_directories

In [4]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config

In [5]:
import os
import urllib.request as request
import zipfile
from WineX import logger
from WineX.utils.common import get_size

In [6]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config


    
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info: \n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")



    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)
  

In [7]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2023-12-08 22:28:12,813: INFO: common] yaml file: config\config.yaml loaded successfully
[2023-12-08 22:28:12,823: INFO: common] yaml file: params.yaml loaded successfully
[2023-12-08 22:28:12,831: INFO: common] yaml file: schema.yaml loaded successfully
[2023-12-08 22:28:12,838: INFO: common] created directory at: artifacts
[2023-12-08 22:28:12,846: INFO: common] created directory at: artifacts/data_ingestion


[2023-12-08 22:28:17,041: INFO: 2366278714] artifacts/data_ingestion/data.zip download! with following info: 
Content-Type: text/html; charset=utf-8
X-Frame-Options: DENY
Set-Cookie: __Host-GAPS=1:QOLRQb8JtBHIng3ELqHcM2pVnoq1Xg:7mRe3LCwbwA0hoV1; Expires=Sun, 07-Dec-2025 21:28:16 GMT; Path=/; Secure; HttpOnly; Priority=HIGH
x-auto-login: realm=com.google&args=service%3Dcds%26continue%3Dhttps://storage.cloud.google.com/wine_data_buckets/winequality-data.zip
x-ua-compatible: IE=edge
Cache-Control: no-cache, no-store, max-age=0, must-revalidate
Pragma: no-cache
Expires: Mon, 01 Jan 1990 00:00:00 GMT
Date: Fri, 08 Dec 2023 21:28:16 GMT
Strict-Transport-Security: max-age=31536000; includeSubDomains
Content-Security-Policy: require-trusted-types-for 'script';report-uri /v3/signin/_/AccountsSignInUi/cspreport
Content-Security-Policy: script-src 'report-sample' 'nonce-U6ZUGGOjZ_71HFnfpmQ7Ng' 'unsafe-inline';object-src 'none';base-uri 'self';report-uri /v3/signin/_/AccountsSignInUi/cspreport;wor

BadZipFile: File is not a zip file

In [1]:
from google.cloud import storage
import requests
import os

# Set your Google Cloud Storage credentials (replace with your actual credentials)
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "/path/to/your/credentials.json"

# Replace with your Google Cloud Storage bucket and file path
bucket_name = "your-bucket-name"
file_path = "your-file-path-in-the-bucket/winequality-data.zip"
local_file_path = "local-path-to-save/winequality-data.zip"

def download_from_gcs(bucket_name, file_path, local_file_path):
    client = storage.Client()
    bucket = client.bucket(bucket_name)
    blob = bucket.blob(file_path)

    # Download the file to a local path
    blob.download_to_filename(local_file_path)
    print(f"File downloaded to {local_file_path}")

# Download the file from Google Cloud Storage
download_from_gcs(bucket_name, file_path, local_file_path)


[2023-12-09 00:09:07,907: WARNING: _metadata] Compute Engine Metadata server unavailable on attempt 1 of 3. Reason: [WinError 10051] A socket operation was attempted to an unreachable network
[2023-12-09 00:09:07,912: WARNING: _metadata] Compute Engine Metadata server unavailable on attempt 2 of 3. Reason: [WinError 10051] A socket operation was attempted to an unreachable network
[2023-12-09 00:09:07,937: WARNING: _metadata] Compute Engine Metadata server unavailable on attempt 3 of 3. Reason: [WinError 10051] A socket operation was attempted to an unreachable network
[2023-12-09 00:09:07,939: WARNING: _default] Authentication failed using Compute Engine authentication due to unavailable metadata server.


DefaultCredentialsError: Your default credentials were not found. To set up Application Default Credentials, see https://cloud.google.com/docs/authentication/external/set-up-adc for more information.